## Notebook to load models and predict for Pakitsoq 2015-2024

### Import Packages

In [ ]:
# Manually add environment's bin dir to the path inside notebook to run tensorboard
import os
#PATH = os.getenv('PATH')
#%env PATH=/home/theghub/naureenk/.conda/envs/crev/bin:$PATH
# Mute TF 2.15 Bug Warnings (Before importing tensorflow) 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from sys import path
# Appending the U-net package folder
path.append("./src/")
import unet
from unet import metrics
from unet import utils
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils, losses, metrics
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
print(f"Tensorflow ver. {tf.__version__}")

### Load Images as Tensorflow-type Dataset

In [ ]:
IMG_SIZE = (512,512)
#function to read image from file
def read_image(image_name):
    image = tf.io.read_file(image_name)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)/255.0
    return image

### Load models

In [ ]:
Pakitsoq_Isuunguata_model = tf.keras.models.load_model('./Pakitsoq_Isuunguata_models/2024-02-06T00-28_54')

In [ ]:
Isuunguata_model = tf.keras.models.load_model('./issunguata_models/2024_02_19T15_46_28')

In [ ]:
Augm_Pakitsoq = tf.keras.models.load_model('./crevasse_data/2024_02_19T23_09_58')

In [ ]:
Pakitsoq_SGD = tf.keras.models.load_model('./crevasse_data/2024_02_20T22_59_29')

In [ ]:
Pakitsoq_Adam = tf.keras.models.load_model('./crevasse_data/2023-11-13T15-43_43')

In [ ]:
output_directory = "./Pakitsoq 2015-2024 Prediction/"
os.makedirs(output_directory, exist_ok=True)

#### 2015

In [ ]:
Pakitsoq_2015 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2015/*.png", shuffle=False)
Pakitsoq_2015 = Pakitsoq_2015.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2015 = Augm_Pakitsoq.predict(Pakitsoq_2015.batch(batch_size=1))
Pakitsoq_SGD_prediction_2015 = Pakitsoq_SGD.predict(Pakitsoq_2015.batch(batch_size=1))
Pakitsoq_Adam_prediction_2015 = Pakitsoq_Adam.predict(Pakitsoq_2015.batch(batch_size=1))
average_prediction_2015 = (Augm_Pakitsoq_prediction_2015 + Pakitsoq_SGD_prediction_2015 + Pakitsoq_Adam_prediction_2015) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2015.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2015[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2015[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2015[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2015[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2015.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2015[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2015[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2015[i, ...], axis=-1)
    ## Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    ## Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2015_Aug_PAK{}.png'.format(i + 1)), format="PNG")

In [ ]:
for i, (image) in enumerate(Pakitsoq_2015.take(24)):
    prediction_image = np.argmax(average_prediction[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2015[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2015[i, ...], axis=-1)
    ## Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)

#### 2016

In [ ]:
Pakitsoq_2016 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2016/*.png", shuffle=False)
Pakitsoq_2016 = Pakitsoq_2016.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2016 = Augm_Pakitsoq.predict(Pakitsoq_2016.batch(batch_size=1))
Pakitsoq_SGD_prediction_2016 = Pakitsoq_SGD.predict(Pakitsoq_2016.batch(batch_size=1))
Pakitsoq_Adam_prediction_2016 = Pakitsoq_Adam.predict(Pakitsoq_2016.batch(batch_size=1))
average_prediction_2016 = (Augm_Pakitsoq_prediction_2016 + Pakitsoq_SGD_prediction_2016 + Pakitsoq_Adam_prediction_2016) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2016.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2016[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2016[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2016[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2016[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2016.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2016[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2016[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2016[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2016_{}.png'.format(i + 1)), format="PNG")

#### 2017

In [ ]:
Pakitsoq_2017 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2017/*.png", shuffle=False)
Pakitsoq_2017 = Pakitsoq_2017.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2017 = Augm_Pakitsoq.predict(Pakitsoq_2017.batch(batch_size=1))
Pakitsoq_SGD_prediction_2017 = Pakitsoq_SGD.predict(Pakitsoq_2017.batch(batch_size=1))
Pakitsoq_Adam_prediction_2017 = Pakitsoq_Adam.predict(Pakitsoq_2017.batch(batch_size=1))
average_prediction_2017 = (Augm_Pakitsoq_prediction_2017 + Pakitsoq_SGD_prediction_2017 + Pakitsoq_Adam_prediction_2017) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2017.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2017[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2017[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2017[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2017[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2017.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2017[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2017[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2017[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2017_{}.png'.format(i + 1)), format="PNG")

#### 2018

In [ ]:
Pakitsoq_2018 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2018/*.png", shuffle=False)
Pakitsoq_2018 = Pakitsoq_2018.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2018 = Augm_Pakitsoq.predict(Pakitsoq_2018.batch(batch_size=1))
Pakitsoq_SGD_prediction_2018 = Pakitsoq_SGD.predict(Pakitsoq_2018.batch(batch_size=1))
Pakitsoq_Adam_prediction_2018 = Pakitsoq_Adam.predict(Pakitsoq_2018.batch(batch_size=1))
average_prediction_2018 = (Augm_Pakitsoq_prediction_2018 + Pakitsoq_SGD_prediction_2018 + Pakitsoq_Adam_prediction_2018) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2018.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2018[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2018[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2018[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2018[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2018.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2018[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2018[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2018[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2018_{}.png'.format(i + 1)), format="PNG")

#### 2019

In [ ]:
Pakitsoq_2019 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2019/*.png", shuffle=False)
Pakitsoq_2019 = Pakitsoq_2019.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2019 = Augm_Pakitsoq.predict(Pakitsoq_2019.batch(batch_size=1))
Pakitsoq_SGD_prediction_2019 = Pakitsoq_SGD.predict(Pakitsoq_2019.batch(batch_size=1))
Pakitsoq_Adam_prediction_2019 = Pakitsoq_Adam.predict(Pakitsoq_2019.batch(batch_size=1))
average_prediction_2019 = (Augm_Pakitsoq_prediction_2019 + Pakitsoq_SGD_prediction_2019 + Pakitsoq_Adam_prediction_2019) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2019.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2019[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2019[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2019[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2019[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2019.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2019[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2019[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2019[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2019_{}.png'.format(i + 1)), format="PNG")

#### 2020

In [ ]:
Pakitsoq_2020 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2020/*.png", shuffle=False)
Pakitsoq_2020 = Pakitsoq_2020.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2020 = Augm_Pakitsoq.predict(Pakitsoq_2020.batch(batch_size=1))
Pakitsoq_SGD_prediction_2020 = Pakitsoq_SGD.predict(Pakitsoq_2020.batch(batch_size=1))
Pakitsoq_Adam_prediction_2020 = Pakitsoq_Adam.predict(Pakitsoq_2020.batch(batch_size=1))
average_prediction_2020 = (Augm_Pakitsoq_prediction_2020 + Pakitsoq_SGD_prediction_2020 + Pakitsoq_Adam_prediction_2020) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2020.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2020[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2020[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2020[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2020[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2020.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2020[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2020[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2020[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2020_{}.png'.format(i + 1)), format="PNG")

#### 2021

In [ ]:
Pakitsoq_2021 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2021/*.png", shuffle=False)
Pakitsoq_2021 = Pakitsoq_2021.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2021 = Augm_Pakitsoq.predict(Pakitsoq_2021.batch(batch_size=1))
Pakitsoq_SGD_prediction_2021 = Pakitsoq_SGD.predict(Pakitsoq_2021.batch(batch_size=1))
Pakitsoq_Adam_prediction_2021 = Pakitsoq_Adam.predict(Pakitsoq_2021.batch(batch_size=1))
average_prediction_2021 = (Augm_Pakitsoq_prediction_2021 + Pakitsoq_SGD_prediction_2021 + Pakitsoq_Adam_prediction_2021) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2021.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2021[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2021[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2021[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2021[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2021.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2021[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2021[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2021[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2021_{}.png'.format(i + 1)), format="PNG")

#### 2022

In [ ]:
Pakitsoq_2022 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2022/*.png", shuffle=False)
Pakitsoq_2022 = Pakitsoq_2022.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)
# Model Predictions
Augm_Pakitsoq_prediction_2022 = Augm_Pakitsoq.predict(Pakitsoq_2022.batch(batch_size=1))
Pakitsoq_SGD_prediction_2022 = Pakitsoq_SGD.predict(Pakitsoq_2022.batch(batch_size=1))
Pakitsoq_Adam_prediction_2022 = Pakitsoq_Adam.predict(Pakitsoq_2022.batch(batch_size=1))
average_prediction_2022 = (Augm_Pakitsoq_prediction_2022 + Pakitsoq_SGD_prediction_2022 + Pakitsoq_Adam_prediction_2022) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2022.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2022[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2022[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2022[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2022[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2022.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2022[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2022[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2022[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2022_{}.png'.format(i + 1)), format="PNG")

#### 2023

In [ ]:
# Load Pakitsoq images as TF dataset
Pakitsoq_2023 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2023/*.png", shuffle=False)
Pakitsoq_2023 = Pakitsoq_2023.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)

# Model Predictions
Augm_Pakitsoq_prediction_2023 = Augm_Pakitsoq.predict(Pakitsoq_2023.batch(batch_size=1))
Pakitsoq_SGD_prediction_2023 = Pakitsoq_SGD.predict(Pakitsoq_2023.batch(batch_size=1))
Pakitsoq_Adam_prediction_2023 = Pakitsoq_Adam.predict(Pakitsoq_2023.batch(batch_size=1))
average_prediction_2023 = (Augm_Pakitsoq_prediction_2023 + Pakitsoq_SGD_prediction_2023 + Pakitsoq_Adam_prediction_2023) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2023.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2023[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2023[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2023[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2023[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2023.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2023[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2023[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2023[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2023_{}.png'.format(i + 1)), format="PNG")

#### 2024

In [ ]:
# Load Pakitsoq images as TF dataset
Pakitsoq_2024 = tf.data.Dataset.list_files("./gris/Pakitsoq April Medians/PNG/8bit/2024/*.png", shuffle=False)
Pakitsoq_2024 = Pakitsoq_2024.map(read_image, num_parallel_calls=tf.data.AUTOTUNE)

# Model Predictions
Augm_Pakitsoq_prediction_2024 = Augm_Pakitsoq.predict(Pakitsoq_2024.batch(batch_size=1))
Pakitsoq_SGD_prediction_2024 = Pakitsoq_SGD.predict(Pakitsoq_2024.batch(batch_size=1))
Pakitsoq_Adam_prediction_2024 = Pakitsoq_Adam.predict(Pakitsoq_2024.batch(batch_size=1))
average_prediction_2024 = (Augm_Pakitsoq_prediction_2024 + Pakitsoq_SGD_prediction_2024 + Pakitsoq_Adam_prediction_2024) / 3
fig, ax = plt.subplots(24, 5, sharex=True, sharey=True, figsize=(50,200))

for i, (image) in enumerate(Pakitsoq_2024.take(24)):
    ax[i][0].imshow(image[..., -1],cmap=plt.cm.gray); ax[i][0].set_title('Image'); ax[i][0].axis('off')
    ax[i][1].imshow(np.argmax(Augm_Pakitsoq_prediction_2024[i,...],axis=-1),cmap=plt.cm.gray); ax[i][1].set_title('Augment_Pakitsoq'); ax[i][1].axis('off')
    ax[i][2].imshow(np.argmax(Pakitsoq_SGD_prediction_2024[i,...],axis=-1),cmap=plt.cm.gray); ax[i][2].set_title('Pakitsoq_SGD'); ax[i][2].axis('off')
    ax[i][3].imshow(np.argmax(Pakitsoq_Adam_prediction_2024[i,...],axis=-1),cmap=plt.cm.gray); ax[i][3].set_title('Pakitsoq_adam'); ax[i][3].axis('off')
    ax[i][4].imshow(np.argmax(average_prediction_2024[i,...],axis=-1),cmap=plt.cm.gray); ax[i][4].set_title('Average Prediction'); ax[i][4].axis('off')
    plt.tight_layout()

In [ ]:
##Iterate over the dataset and predictions
for i, (image) in enumerate(Pakitsoq_2024.take(24)):
    prediction_image = np.argmax(Augm_Pakitsoq_prediction_2024[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_SGD_prediction_2024[i, ...], axis=-1)
    #prediction_image = np.argmax(Pakitsoq_Adam_prediction_2024[i, ...], axis=-1)
    # Convert to 8-bit image
    prediction_image = prediction_image.astype(np.uint8)
    ## counting the number of pixels 
    #number_of_bg_pix = np.sum(prediction_image == 0) 
    number_of_crevasse_pix = np.sum(prediction_image == 1) 
    #number_of_stream_pix = np.sum(prediction_image == 2)
    #print('Number of black pixels:', number_of_bg_pix)
    print(number_of_crevasse_pix)
    #print('Number of stream/lake pixels:', number_of_stream_pix)
    # Save each prediction image separately
    img = Image.fromarray(prediction_image, mode='L')  # 'L' mode for 8-bit images (grayscale)
    img.save(os.path.join(output_directory, 'Pakitsoq_2024_{}.png'.format(i + 1)), format="PNG")